In [1]:
import pandas as pd
import inro.modeller as m

mm = m.Modeller()
eb = mm.emmebank
sc_am = eb.scenario(10)
ntwk = sc_am.get_network()

## Link Splitting Code below

In [12]:
maxNode = 0

In [13]:
for node in ntwk.nodes():
    #print(type(int(node.id)),type(maxNode))
    if int(node.id) > maxNode:
        maxNode = int(node.id)

In [14]:
next_available = maxNode + 1

In [15]:
next_available

12710

In [37]:
linkList = []
for link in ntwk.links():
    if link.type == 100 and link.length > 0.025:
        linkList.append({"i": link.i_node, "j": link.j_node})

In [38]:
linkdf = pd.DataFrame.from_dict(linkList)

In [39]:
linkdf

""


In [36]:
del_list = []
for index, row in linkdf.iterrows():
    a_split = False
    for link in del_list:
        if link["i"] == row.j.id and link["j"] == row.i.id:
            a_split = True
    if not a_split:
        ntwk.split_link(row.i.id,row.j.id,str(next_available), include_reverse = True)
        del_list.append({"i": row.i.id, "j": row.j.id})
        next_available = next_available + 1

In [20]:
del_list

[{'i': '10001', 'j': '10007'},
 {'i': '10003', 'j': '10015'},
 {'i': '10004', 'j': '10005'},
 {'i': '10004', 'j': '10023'},
 {'i': '10005', 'j': '10006'},
 {'i': '10005', 'j': '10008'},
 {'i': '10006', 'j': '10010'},
 {'i': '10006', 'j': '10012'},
 {'i': '10008', 'j': '10012'},
 {'i': '10008', 'j': '10013'},
 {'i': '10009', 'j': '10016'},
 {'i': '10011', 'j': '10018'},
 {'i': '10011', 'j': '10020'},
 {'i': '10012', 'j': '10019'},
 {'i': '10013', 'j': '10019'},
 {'i': '10013', 'j': '10025'},
 {'i': '10014', 'j': '10018'},
 {'i': '10014', 'j': '10026'},
 {'i': '10015', 'j': '10047'},
 {'i': '10017', 'j': '10022'},
 {'i': '10018', 'j': '10035'},
 {'i': '10019', 'j': '10030'},
 {'i': '10020', 'j': '10158'},
 {'i': '10021', 'j': '10027'},
 {'i': '10021', 'j': '10161'},
 {'i': '10023', 'j': '10025'},
 {'i': '10023', 'j': '10036'},
 {'i': '10024', 'j': '10031'},
 {'i': '10027', 'j': '10061'},
 {'i': '10028', 'j': '10030'},
 {'i': '10028', 'j': '10041'},
 {'i': '10029', 'j': '10038'},
 {'i': '

In [40]:
sc_am.publish_network(ntwk)

##Centroid Generation begins here

In [37]:
sc_am = eb.scenario(11)
ntwk = sc_am.get_network()

In [38]:
masterNode = []
for node in ntwk.nodes():
    masterNode.append({"ID": node.id, "x": node.x, "y": node.y})

In [39]:
mnodedf = pd.DataFrame.from_dict(masterNode)

In [40]:
for centroid in ntwk.centroids():
    x = centroid.x
    y = centroid.y
    mnodedf["dist"] = ((x - mnodedf["x"]).pow(2) +(y - mnodedf["y"]).pow(2)).pow(0.5)
    sortdf = mnodedf.sort("dist")
    match = False
    i = 0
    while not match:
        node = ntwk.node(sortdf["ID"].values[i])
        
        if node.is_centroid == False:
            
            ped_network_link = False
            
            for link in node.incoming_links():
                if link.type == 100:
                    ped_network_link = True
            
            if ped_network_link:
                match = True
                matchNode = node.id
            else:
                i = i + 1
        else:
            i = i + 1
            
    link1 = ntwk.create_link(centroid.id, matchNode, modes = "c")
    link1["type"] = 999
    link2 = ntwk.create_link(matchNode, centroid.id,modes = "c")
    link2["type"] = 999

In [41]:
sc_am.publish_network(ntwk)